In [ ]:
pip install pillow


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
# Steganography Project - Hiding Information in Image (LSB Method)

from PIL import Image


In [ ]:
# Convert message to binary
def text_to_binary(message):
    return ''.join([format(ord(char), '08b') for char in message])

In [ ]:
# Convert binary to text
def binary_to_text(binary_str):
    chars = [binary_str[i:i+8] for i in range(0, len(binary_str), 8)]
    return ''.join([chr(int(char, 2)) for char in chars])


In [ ]:
# Encode the message into image
def encode_image(image_path, message, output_path):
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    binary_message = text_to_binary(message) + '1111111111111110'  # Delimiter
    data_index = 0
    new_pixels = []

    for pixel in list(image.getdata()):
        r, g, b = pixel
        new_rgb = []

        for color in (r, g, b):
            if data_index < len(binary_message):
                lsb = binary_message[data_index]
                new_color = (color & ~1) | int(lsb)
                new_rgb.append(new_color)
                data_index += 1
            else:
                new_rgb.append(color)

        new_pixels.append(tuple(new_rgb))

    if data_index < len(binary_message):
        raise ValueError("Message is too long to encode in this image.")

    image.putdata(new_pixels)
    image.save(output_path)
    print("✅ Message encoded and saved to", output_path)

In [ ]:
# Decode the hidden message from image
def decode_image(image_path):
    image = Image.open(image_path)
    binary_data = ''
    for pixel in list(image.getdata()):
        for color in pixel[:3]:
            binary_data += str(color & 1)
              # Split binary data by 8 bits
    all_bytes = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    
    # Stop at delimiter
    message = ''
    for byte in all_bytes:
        if byte == '11111110':  # Delimiter
            break
        message += chr(int(byte, 2))
    return message

In [ ]:
# --- Test the Steganography ---
# Step 1: Encode
input_image = "download.jpeg"      # Use your image here
output_image = "encoded.png"
secret_message = "I am AYUSH KUMAR DEWANGAN "

encode_image(input_image, secret_message, output_image)

# Step 2: Decode
decoded_message = decode_image(output_image)
print("🔍 Decoded message:", decoded_message)

✅ Message encoded and saved to encoded.png
🔍 Decoded message: I am AYUSH KUMAR DEWANGAN ÿ
